# Day 5: Print Queue

Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

---

## The Problem

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation `X|Y` means that if both page number `X` and page number `Y` are to be produced as part of an update, page number `X` must be printed at some point **before** page number `Y`.

The Elf has for you both:
- **Page ordering rules** and
- **Pages to produce in each update** (your puzzle input).

Your task is to determine whether each update has the pages in the correct order.

---

## Example Input

### Page Ordering Rules
```
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13
```

### Updates
```
75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
```

---

## Example Explanation

### First Update: `75,47,61,53,29`
This update is in the correct order:
1. **75** is first because it precedes all other pages (`75|47`, `75|61`, `75|53`, `75|29`).
2. **47** is second because it follows `75` (`75|47`) and precedes all other pages it relates to (`47|61`, `47|53`, `47|29`).
3. **61** is in the middle because it follows `75` and `47` (`75|61`, `47|61`) and precedes `53` and `29` (`61|53`, `61|29`).
4. **53** is fourth because it precedes `29` (`53|29`).
5. **29** is last.

### Second and Third Updates
Both are also in the correct order based on the relevant rules.

### Fourth Update: `75,97,47,61,53`
This update is **not** in the correct order because `75` appears before `97`, which violates the rule `97|75`.

### Fifth Update: `61,13,29`
This update is **not** in the correct order because `29` appears before `13`, which violates the rule `29|13`.

### Last Update: `97,13,75,29,47`
This update breaks several rules and is not in the correct order.

---

## Middle Page Numbers of Correctly-Ordered Updates

For correctly-ordered updates:
1. `75,47,61,53,29` → Middle page: **61**
2. `97,61,53,29,13` → Middle page: **53**
3. `75,29,13` → Middle page: **29**

Adding these together gives:
```
61 + 53 + 29 = 143
```

---

## Task

Determine which updates are already in the correct order. **What do you get if you add up the middle page number from those correctly-ordered updates?**

In [1]:
from typing import Tuple
from dataclasses import dataclass
import numpy as np

@dataclass
class Rule:
    X: int
    Y: int

    @classmethod
    def from_string(cls, s: str):
        x, y = s.strip().split("|")
        return cls(int(x), int(y))

    def fulfilled(self, page_numbers: np.ndarray[int]) -> bool:
        # true when X|Y is not present or X is present before Y
        x_index = np.nonzero(page_numbers == self.X)[0]
        y_index = np.nonzero(page_numbers == self.Y)[0]
        if len(x_index) == 0 or len(y_index) == 0:
            return True
        if x_index.min() < y_index.min():
            return True
        return False
    
    def fix_order(self, page_numbers: np.ndarray[int]) -> None:
        # in-place numpy array manipulation
        x_index = np.nonzero(page_numbers == self.X)[0]
        y_index = np.nonzero(page_numbers == self.Y)[0]
        if len(x_index) == 0 or len(y_index) == 0:
            return
        if x_index.min() > y_index.min():
            # swap the two elements with pythonic tuple unpacking
            page_numbers[x_index], page_numbers[y_index] = page_numbers[y_index], page_numbers[x_index]
        


@dataclass
class Update:
    page_numbers: np.ndarray[int]

    @classmethod
    def from_string(cls, s: str):
        return cls(np.fromstring(s.strip(), dtype=int, sep=","))
    
    def is_valid(self, rules: list[Rule]) -> bool:
        for rule in rules:
            if not rule.fulfilled(self.page_numbers):
                return False
        return True
    
    def fix_order(self, rules: list[Rule]) -> "Update":
        while not self.is_valid(rules):
            for rule in rules:
                rule.fix_order(self.page_numbers)  # in-place manipulation
        return self
        
    
    def get_middle_page_number(self) -> int:
        return self.page_numbers[len(self.page_numbers) // 2]


def read_ordering_rules_and_updates(filepath: str) -> Tuple[list[Rule], list[Update]]:
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.readlines()
    # rules are anything before the first empty line
    empty_line_index = content.index("\n")
    rules = [Rule.from_string(line) for line in content[:empty_line_index]]
    updates = [Update.from_string(line) for line in content[empty_line_index+1:]]
    return rules, updates


ordering_rules, updates = read_ordering_rules_and_updates("./example.txt")
valid_updates = [update for update in updates if update.is_valid(ordering_rules)]
middle_sum = sum(update.get_middle_page_number() for update in valid_updates)
print(f"{middle_sum=}, {valid_updates=}")

ordering_rules, updates = read_ordering_rules_and_updates("./input.txt")
valid_updates = [update for update in updates if update.is_valid(ordering_rules)]
middle_sum = sum(update.get_middle_page_number() for update in valid_updates)
print(f"{middle_sum=}, {len(valid_updates)=}")

middle_sum=143, valid_updates=[Update(page_numbers=array([75, 47, 61, 53, 29])), Update(page_numbers=array([97, 61, 53, 29, 13])), Update(page_numbers=array([75, 29, 13]))]
middle_sum=5374, len(valid_updates)=96


# Part Two

While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. 

---

## Example

Here are the three incorrectly-ordered updates and their correct orderings:

1. `75,97,47,61,53` becomes `97,75,47,61,53`.
2. `61,13,29` becomes `61,29,13`.
3. `97,13,75,29,47` becomes `97,75,47,29,13`.

---

## Task

After taking only the incorrectly-ordered updates and ordering them correctly, their **middle page numbers** are:

- `97,75,47,61,53` → Middle page: **47**
- `61,29,13` → Middle page: **29**
- `97,75,47,29,13` → Middle page: **47**

Adding these together produces:
```
47 + 29 + 47 = 123
```

---

## Objective

Find the updates which are not in the correct order. **What do you get if you add up the middle page numbers after correctly ordering just those updates?**

In [2]:
ordering_rules, updates = read_ordering_rules_and_updates("./example.txt")
corrected_updates = [
    update.fix_order(ordering_rules)
    for update in updates
    if not update.is_valid(ordering_rules)
]
middle_sum = sum(update.get_middle_page_number() for update in corrected_updates)
print(f"{middle_sum=}, {corrected_updates=}")

ordering_rules, updates = read_ordering_rules_and_updates("./input.txt")
corrected_updates = [
    update.fix_order(ordering_rules)
    for update in updates
    if not update.is_valid(ordering_rules)
]
middle_sum = sum(update.get_middle_page_number() for update in corrected_updates)
print(f"{middle_sum=}, {len(corrected_updates)=}")

middle_sum=123, corrected_updates=[Update(page_numbers=array([97, 75, 47, 61, 53])), Update(page_numbers=array([61, 29, 13])), Update(page_numbers=array([97, 75, 47, 29, 13]))]
middle_sum=4260, len(corrected_updates)=88
